In [ ]:
import Pyro4

camera_0 = Pyro4.Proxy("PYRO:SimulatedCamera@camera_0:8000")
camera_0.enable()
print(camera_0.get_exposure_time())

laser_0 = Pyro4.Proxy("PYRO:SimulatedLightSource@light_0:9000")
laser_0.enable()
print(laser_0.power)